<a href="https://colab.research.google.com/github/KarolMc/msc_dissertation/blob/main/feature_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import joblib

import pandas as pd
import numpy as np

# Import SVC from sklearn.svm and accuracy_score from sklearn.metrics
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import make_scorer

import random

random_seed = random.seed(42)

from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt

In [ ]:
X_train = pd.read_csv('Data/X_train_SMOV.csv', index_col=[0])
y_train = pd.read_csv('Data/y_train_SMOV.csv', index_col=[0])
X_test = pd.read_csv('Data/X_test.csv', index_col=[0])
y_test = pd.read_csv('Data/y_test.csv', index_col=[0])

train_features = X_train.columns

In [ ]:
diabetes_data = X_train

In [ ]:
diabetes_data.head()

,SEQN,Exam_status,Age_Years,Household_size,Family_size,Family_income_poverty_ratio,Folic_acid_nmol_l,Albumin_urine_ug_ml,Albumin_urine_mg_l,Creatinine_unrine_umol_l,...,BPCuff_Large,BPCuff_Thigh,BPCuff_infant,Pulse_irregular,Pulse_regular,BMI_Healthy,BMI_Obese,BMI_Overweight,BMI_Underweight,BMI_unknown
0,71180,2.0,57.0,7.0,7.0,1.80,0.68,5.6,5.6,7248.8,...,0,0,0,0,1,1,0,0,0,0
1,66825,2.0,8.0,4.0,4.0,1.56,0.68,13.6,13.6,10342.8,...,0,0,0,0,1,0,0,0,1,0
2,83409,2.0,43.0,7.0,6.0,0.54,0.68,24.2,24.2,6541.6,...,1,0,0,0,1,0,1,0,0,0
3,83957,2.0,0.0,4.0,4.0,0.49,0.68,8.4,8.4,7072.0,...,0,0,0,0,1,0,0,0,0,1
4,70223,2.0,47.0,7.0,1.0,0.79,0.68,113.0,113.0,33503.6,...,1,0,0,1,0,0,1,0,0,0


In [ ]:
diabetes_labels = y_train

diabetes_labels = diabetes_labels.rename({'0': 'Diabetes_status'}, axis = 1)
diabetes_labels.head()

,Diabetes_status
0,0
1,0
2,1
3,0
4,0


In [ ]:
diabetes_training_data = pd.concat([diabetes_data, diabetes_labels], axis = 1)

diabetes_training_data.shape

(75864, 211)

In [ ]:
diabetes_training_data.head()

,SEQN,Exam_status,Age_Years,Household_size,Family_size,Family_income_poverty_ratio,Folic_acid_nmol_l,Albumin_urine_ug_ml,Albumin_urine_mg_l,Creatinine_unrine_umol_l,...,BPCuff_Thigh,BPCuff_infant,Pulse_irregular,Pulse_regular,BMI_Healthy,BMI_Obese,BMI_Overweight,BMI_Underweight,BMI_unknown,Diabetes_status
0,71180,2.0,57.0,7.0,7.0,1.80,0.68,5.6,5.6,7248.8,...,0,0,0,1,1,0,0,0,0,0
1,66825,2.0,8.0,4.0,4.0,1.56,0.68,13.6,13.6,10342.8,...,0,0,0,1,0,0,0,1,0,0
2,83409,2.0,43.0,7.0,6.0,0.54,0.68,24.2,24.2,6541.6,...,0,0,0,1,0,1,0,0,0,1
3,83957,2.0,0.0,4.0,4.0,0.49,0.68,8.4,8.4,7072.0,...,0,0,0,1,0,0,0,0,1,0
4,70223,2.0,47.0,7.0,1.0,0.79,0.68,113.0,113.0,33503.6,...,0,0,1,0,0,1,0,0,0,0


In [ ]:
diabetes_training_data['Diabetes_status'].value_counts()

0    25288
1    25288
2    25288
Name: Diabetes_status, dtype: int64

In [ ]:
features = list(diabetes_training_data.columns)

In [ ]:
!pip install statsmodels

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 60.8 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.8/233.8 kB 45.8 MB/s eta 0:00:00


In [ ]:
# load statmodels functions
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# compute the vif for all given features
def compute_vif(considered_features):
    
    X = diabetes_training_data[considered_features]
    # the calculation of variance inflation requires a constant
    X['intercept'] = 1
    
    # create dataframe to store vif values
    vif = pd.DataFrame()
    vif["Variable"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif[vif['Variable']!='intercept']
    return vif

In [ ]:
vifs_df = compute_vif(features)

/usr/local/lib/python3.9/dist-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/usr/local/lib/python3.9/dist-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [ ]:
vif_df = vifs_df.sort_values(by=['VIF'], ascending = False)
inf_vif_df = vif_df.loc[vif_df['VIF']== np.inf]

inf_vif_df
# vif_df = vif_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# vif_df.head()
# vif_df = vif_df.dropna(subset=["VIF"], how="all", inplace=True)
# vif_df = vif_df[vif_df.VIF < inf]
# vif_df.head(30)

,Variable,VIF
128,AdultEd_Under_20,inf
7,Albumin_urine_ug_ml,inf
8,Albumin_urine_mg_l,inf
135,Marital_Status_Child,inf


In [ ]:
high_vif_list = inf_vif_df['Variable'].to_list()

In [ ]:
high_vif_list

['AdultEd_Under_20',
 'Albumin_urine_ug_ml',
 'Albumin_urine_mg_l',
 'Marital_Status_Child']

In [ ]:
dropped_onehots = ['No_gum_disease', 'BMI_Healthy', 'HEPB_negative', 'HEPD_negative', 'No_decayed_teeth', 
                   'BPCuff_infant', 'Pulse_regular', 'HEPA_negative', 'Albumin_urine_mg_l', 'Household_income_25-35k',
                   'AdultEd_Under_20', 'ChildEd_Adult', 'Marital_Status_Child', 'Family_income_25-35k', 'US_citizen', 
                   'Race_Non_Hispanic_White', 'HPV_not_applicable', 'HCV_negative', 'chlamydia_negative', 'HSV1_negative', 
                   'HSV2_negative', 'Born_US', 'No_military_service','Female', 'Not_pregnant', 'URXUCR']

In [ ]:
diabetes_training_data_reduced = diabetes_training_data.drop(dropped_onehots, axis = 1)

In [ ]:
features = list(diabetes_training_data_reduced.columns)

In [ ]:
# compute the vif for all given features
def updated_compute_vif(considered_features):
    
    X = diabetes_training_data_reduced[considered_features]
    # the calculation of variance inflation requires a constant
    X['intercept'] = 1
    
    # create dataframe to store vif values
    vif = pd.DataFrame()
    vif["Variable"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif[vif['Variable']!='intercept']
    return vif

In [ ]:
vifs_df = compute_vif(features)

/tmp/ipykernel_2480/2718942932.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['intercept'] = 1
/usr/local/lib/python3.9/dist-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


In [ ]:
vifs_df = vifs_df.sort_values(by=['VIF'], ascending = False)
vifs_df.tail(30)

,Variable,VIF
161,Race_Other_Hispanic,1.299182
129,Marital_Status_Living_with_Partner,1.286366
133,Marital_Status_Widowed,1.286167
90,Potassium_mmol_L,1.258069
127,ChildEd_None_KG,1.224669
126,ChildEd_HS_Grad,1.212108
103,Served_Military,1.192893
174,Gum_disease,1.176509
173,Decayed_teeth,1.176108
71,Urinary_Uranium_ug_l,1.175960


In [ ]:
vifs_df.head(50)

,Variable,VIF
45,Total_Cholesterol_mmol_l,152292.957363
44,Total_Cholesterol_mg_dl,152225.531594
46,Direct_HDL_Cholesterol_mmol_l,19057.717210
47,Direct_HDL_Cholesterol_mg_dl,19056.772647
57,Blood_lead_umol_l,17920.172988
56,Blood_Lead_ug_l,17906.035485
12,Segmented_neutrophils_percent,2865.888966
10,Lymphocyte_percent,2378.612699
29,Urinary_perchlorate_ng_ml,1773.840334
30,Urinary_nitrate_ng_ml,1773.672207


In [ ]:
# CORRELATIONS

In [ ]:
diabetes_training_data_reduced['3'].value_counts()

0    75839
1       25
Name: 3, dtype: int64

In [ ]:
to_drop = ['2', '3', 'Total_Cholesterol_mg_dl', 'Direct_HDL_Cholesterol_mg_dl', 'Blood_Lead_ug_l', 'Total_Blood_Mercury_ug_l',
           'Blood_Cadmium_ug_l']

In [ ]:
high_vif_to_drop = ['Segmented_neutrophils_percent', 'Lymphocyte_percent', 'Monocyte_percent', 'Eosinophils_percent', 'BPCuff_Adult', 'Family_size', 'Family_income_greater100k', 'Family_income_75-100k', 'Family_income_55-65k', 'Family_income_35-45k', 'Family_income_65-75k', 'Family_income_45-55k', 'Family_income_20-25k', 'Family_income_10-15k', 'Family_income_5-10k', 'Family_income_15-20k', 'Family_income_less4999', 'Cholesterol_mmol_L', '25_hydroxyvitamin_D3_nmol_L' ]

In [ ]:
diabetes_training_data_reduced = diabetes_training_data_reduced.drop(to_drop, axis = 1)

diabetes_training_data_reduced = diabetes_training_data_reduced.drop(high_vif_to_drop, axis = 1)

diabetes_training_data_reduced = diabetes_training_data_reduced.drop(['URDUMMAL', 'Exam_status'], axis = 1) 

In [ ]:
diabetes_training_data_reduced.shape

(75864, 157)

In [ ]:
reduced_columns = list(diabetes_training_data_reduced.columns)

reduced_columns.pop()

reduced_columns

['SEQN',
 'Age_Years',
 'Household_size',
 'Family_income_poverty_ratio',
 'Folic_acid_nmol_l',
 'Albumin_urine_ug_ml',
 'Creatinine_unrine_umol_l',
 'White_Blood_Cell_Count',
 'Basophils_percent',
 'Lymphocyte_number',
 'Monocyte_number',
 'Segmented_neutrophils_number',
 'Eosinophils_number',
 'Basophils_number',
 'Red_Cell_count',
 'Haemoglobin_g_dl',
 'Haematocrit_percent',
 'Mean_Cell_Volume_fl',
 'Mean_Cell_Haemoglobin_pg',
 'Mean_Cell_Haemoglobin_Concentration_g_dl',
 'Red_Cell_Distribution_Width_Percent',
 'Platelet_Count',
 'Mean_Platelet_Volumne_fl',
 'Urinary_perchlorate_ng_ml',
 'Urinary_nitrate_ng_ml',
 'Urinary_Thiocyanate_ng_ml',
 'Urinary_Total_Arsenic_ug_l',
 'Urinary_arsenous_acid_ug_l',
 'Urinary_Arsenic_Acid_ug_l',
 'Urinary_Arsenobetaine_ug_l',
 'Urinary_Arsenocholine_ug_l',
 'Urinary_DimethylArsinic_Acid_ug_l',
 'Urinary_Monomethylarsonic_acid_ug_l',
 'Urinary_Trimethylarsine_Oxide_ug_l',
 'Klotho_pg_ml',
 'Urinary_Iodine_ug_l',
 'Cotinine_ng_ml',
 'Total_Choleste

In [ ]:
X_train_reduced = X_train[reduced_columns]

X_train_reduced.shape

(75864, 156)

In [ ]:
X_test_reduced = X_test[reduced_columns]

X_test_reduced.shape

(12178, 156)

In [ ]:
X_train_reduced.to_csv('Data/X_train_reduced.csv')

In [ ]:
X_test_reduced.to_csv('Data/X_test_reduced.csv')